<a href="https://colab.research.google.com/github/manumeehl/trajectories/blob/main/trajectories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install tesseract-ocr
!pip install opencv-python-headless pytesseract pytube

In [ ]:
from google.colab.patches import cv2_imshow as show
from pytube import YouTube
import pandas as pd
import pytesseract
import math
import cv2
import re

In [ ]:
def get_video(url, path):
  vid = YouTube(url)
  print(f"Downloading Video '{vid.streams[0].title}'")
  vid.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path=path, filename='launch.mp4')
  print('Download complete')

In [484]:
def custom_crop(frame):
  height, width, _ = frame.shape
  return frame[height-140:height,0:1920]

def crop_roi(frame, roi):
  ''' Extract from a list of predefined regions of interest (ROI) '''
  height, width, _ = frame.shape

  if roi == 'Clock':
    return frame[height-110:height-20,int(width/2-200):int(width/2+200)]
  if roi == 'Stage 1':
    return frame[height-80:height-40,50:300]
  elif roi == 'Stage 2':
    return frame[height-120:height-20,width-300:width]

In [487]:
def clock_to_secs(clockstring):
  ''' Convert a mission clock string to an integer second value '''
  tense = -1 if clockstring[:2] == 't-' else 1
  # Remove the T+- sign
  clockstring = clockstring [2:]
  # Get the seconds
  units = clockstring.split(':')
  hours = int(units[0])
  minutes = int(units[1])
  seconds = int(units[2])

  return hours * 3600 + minutes * 60 + seconds

def frame_to_elapsed(framepos):
  # Calculate the exact time since liftoff based on the current frame
  return  round((framepos - liftoff_frame) / fps, 2)

In [489]:
def get_trajectory_from_string(string):
  ''' Extract altitude and velocity from a given string '''
  string = string.lower()

  # Apply a blacklist
  blacklist = ['stage 1', 'stage 2', 'falcon 9', 'starlink']
  for w in blacklist:
    string = string.replace(w, '')

  # Find all numbers and floats in the string
  numbers = re.findall(r'\d+(?:\.\d+)?', string)
  numbers = list(set([float(x) for x in numbers]))
  if numbers:
    return {
        'Altitude': min(numbers),
        'Velocity': max(numbers)
    }
  else:
    return {}

def get_clock_from_string(string):
  ''' Extract mission clock, altitude and velocity from a given string '''
  string = string.lower()

  # Find the mission clock (T+00:00:00)
  mission_clock = re.findall(r't[+-]\d{2}:\d{2}:\d{2}', string)

  if mission_clock:
    return {'Clock': mission_clock[0]}
  else:
    return {}

In [490]:
def await_liftoff():
  ''' Identify the frame at which liftoff occurs '''
  framepos = 0
  interval = 300
  clock = ''
  # Await first mission clock appearance
  while vid.isOpened():
    vid.set(cv2.CAP_PROP_POS_FRAMES, int(framepos))
    framepos += interval

    ret, frame = vid.read()
    data = extract_data(frame)

    if data['Clock']:
      clock = data['Clock']
      break

  # Now wait until the next full second
  interval = 1
  while vid.isOpened():
    vid.set(cv2.CAP_PROP_POS_FRAMES, int(framepos))
    framepos += interval

    ret, frame = vid.read()
    data = extract_data(frame)

    if data['Clock'] and data['Clock'] != clock:
      framepos += int(data['Elapsed'] * fps)
      return framepos


In [ ]:
def floor_tenth(number):
    return math.floor(number * 10) / 10

def process_launch(vid, liftoff_frame):
  interval = 1

  framepos = liftoff_frame

  monitor = pd.DataFrame(columns = ['Clock', 'Elapsed', 'Altitude', 'Velocity'])

  curr_elapsed = 0.0

  while vid.isOpened():
    vid.set(cv2.CAP_PROP_POS_FRAMES, int(framepos))
    elapsed = frame_to_elapsed(framepos)
    framepos += interval

    if math.floor(elapsed) >= curr_elapsed:
      # Fetch for this timeframe
      ret, frame = vid.read()
      data = extract_data(frame)

      if data['Clock']:
        data['Elapsed'] = math.floor(elapsed)
        monitor.loc[len(monitor)] = data
        curr_elapsed += 1

    if framepos > liftoff_frame + 100:
      break

  return monitor

def process_launch_2(vid, liftoff_frame):
  framepos = liftoff_frame
  trajectory = pd.DataFrame(columns = ['Clock', 'Elapsed', 'Altitude', 'Velocity'])

  while vid.isOpened():
    vid.set(cv2.CAP_PROP_POS_FRAMES, int(framepos))
    ret, frame = vid.read()
    data = extract_data(frame)

    if 'Clock' in data.keys() and 'Velocity' in data.keys() and 'Altitude' in data.keys():
      # Save data
      trajectory.loc[len(trajectory)] = data
      # Skip to next second
      framepos += int(fps - ((framepos - liftoff_frame) % fps)) + 3
    else:
      framepos += 3
    if framepos > liftoff_frame + 1000:
      break

  return trajectory


In [ ]:
url = 'https://www.youtube.com/watch?v=agYuEAkEljw'
get_video(url, 'launches')

Download complete


In [ ]:
vidpath = 'launches/launch.mp4'
vid = cv2.VideoCapture(vidpath)
fps = vid.get(cv2.CAP_PROP_FPS)

print('Awaiting liftoff')
liftoff_frame = await_liftoff()
print('Liftoff registered, now monitoring launch')

#vid.set(cv2.CAP_PROP_POS_FRAMES, int(liftoff_frame))
#ret, frame = vid.read()
#show(frame)

Awaiting liftoff
Liftoff registered, now monitoring launch


In [493]:
def extract_data(frame):
  # Extract telemetry of Stage 1
  crop = crop_roi(frame, 'Stage 1')
  gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
  inverted = cv2.bitwise_not(gray)
  trajectory = get_trajectory_from_string(
      pytesseract.image_to_string(inverted, config='--psm 11'))

  # Extract telemetry of the Mission Clock
  crop = crop_roi(frame, 'Clock')
  gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
  inverted = cv2.bitwise_not(gray)
  clock = get_clock_from_string(
      pytesseract.image_to_string(inverted))

  return {**clock, **trajectory}

def test():
  framepos = 14090

  for i in range(10):
    vid.set(cv2.CAP_PROP_POS_FRAMES, int(framepos))
    ret, frame = vid.read()
    data = extract_data(frame)

    print(data)

    framepos +=30

In [503]:
''' Data post-processing '''
def smooth_curve(curve):
  ''' Correct decimal errors of the OCR algorithm '''
  for i in range(len(curve[1:])):
    if curve[i+1] / curve[i] > 2:
      curve[i+1] /= 10
    elif curve[i+1] / curve[i] < 0.5:
      curve[i+1] *= 10
    return curve

In [ ]:
trajectory = process_launch_2(vid, liftoff_frame)
trajectory['Altitude'] = smooth_curve(trajectory['Altitude'].values.tolist())
trajectory.to_csv('trajectory2.csv')

{'Clock': 't+00:00:00', 'Altitude': 0.1, 'Velocity': 4.0}
{'Clock': 't+00:00:01', 'Altitude': 1.0, 'Velocity': 1.0}
{'Clock': 't+00:00:02', 'Altitude': 0.1, 'Velocity': 9.0}
{'Clock': 't+00:00:03', 'Altitude': 0.1, 'Velocity': 17.0}
{'Clock': 't+00:00:04', 'Altitude': 0.1, 'Velocity': 19.0}
{'Clock': 't+00:00:05', 'Altitude': 0.1, 'Velocity': 30.5}
{'Clock': 't+00:00:06', 'Altitude': 0.1, 'Velocity': 42.0}
{'Clock': 't+00:00:08', 'Altitude': 0.1, 'Velocity': 69.0}
{'Clock': 't+00:00:08', 'Altitude': 0.1, 'Velocity': 72.0}
{'Clock': 't+00:00:09', 'Altitude': 0.2, 'Velocity': 89.0}
{'Clock': 't+00:00:10', 'Altitude': 0.2, 'Velocity': 105.0}
{'Clock': 't+00:00:11', 'Altitude': 0.2, 'Velocity': 120.0}
{'Clock': 't+00:00:12', 'Altitude': 0.3, 'Velocity': 144.0}
{'Clock': 't+00:00:13', 'Altitude': 0.3, 'Velocity': 153.0}
